In [1]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [11]:
import requests
import pymysql

# API details
api_key = "9vwTpZnV8T3jHXHKfpAU116yzoSYHMJJ8iimvq7w"
url = f"https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key={api_key}"

# Fetch data from API
data = requests.get(url).json()

# Connect to MySQL using PyMySQL
conn = pymysql.connect(
    host="127.0.0.1",
    user="root",
    password="Laptop@321",
    database="tennis"
)
cursor = conn.cursor()

# Loop through the rankings and then through competitor_rankings
for ranking in data.get("rankings", []):
    for competitor_ranking in ranking.get("competitor_rankings", []):
        c = competitor_ranking["competitor"]

        # Insert or update Competitors table
        cursor.execute("""
            INSERT INTO Competitor (competitor_id, name, country, country_code, abbreviation)
            VALUES (%s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                name = VALUES(name),
                country = VALUES(country),
                country_code = VALUES(country_code),
                abbreviation = VALUES(abbreviation)
        """, (c["id"], c["name"], c.get("country", ""), c.get("country_code", ""), c.get("abbreviation", "")))

        # Insert into Competitor_Rankings table
        cursor.execute("""
            INSERT INTO Competitor_Rankings (`rank`, movement, points, competitions_played, competitor_id)
            VALUES (%s, %s, %s, %s, %s)
        """, (competitor_ranking["rank"], competitor_ranking.get("movement", 0), competitor_ranking.get("points", 0), competitor_ranking.get("competitions_played", 0), c["id"]))

# Commit and close
conn.commit()
cursor.close()
conn.close()
